In [1]:
import pandas as pd
import datetime
import re

path = '/Users/sachin/Code/week4/project2_git/DScience_P2_SvSaAn/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

df_list = [orderlines, orders, brands, products]
files = ['orderlines','orders','brands','products']

In [2]:
(
orderlines
    .sort_values(('product_quantity'), ascending=False)
    .groupby('sku')
    .head(50)
)

,id,id_order,product_id,product_quantity,sku,unit_price,date
68712,1254032,358747,0,999,SEV0028,19.99,2017-05-24 14:51:58
53860,1228150,346221,0,999,APP1190,55.99,2017-04-14 21:50:52
57796,1234924,349475,0,800,KIN0137,7.49,2017-04-25 09:59:00
57306,1234111,349133,0,555,APP0665,70.99,2017-04-24 10:20:13
40813,1204788,335057,0,201,THU0029,80.99,2017-03-14 15:25:53
...,...,...,...,...,...,...,...
101937,1313510,387300,0,1,WIK0008,99.99,2017-08-08 00:27:13
101942,1313518,386339,0,1,PAC0571,168.58,2017-08-08 00:45:06
101943,1313520,387304,0,1,REP0313,139.99,2017-08-08 00:46:39
101948,1313529,387307,0,1,SPE0159,39.99,2017-08-08 00:57:50


## Missing values

We will define a function that checks how many missing values are in each column of a dataframe:

In [3]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

In [4]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


File name: brands
Missing values:
short    0
long     0
dtype: int64


File name: products
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64




## Duplicates

We will do the same with the duplicates: create a function to detect them in each dataframe, and then iterate through all the dataframes.

In [5]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

In [6]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0


File name: brands
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0


File name: products
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  8746




## Data types

In [7]:
for i in range(len(df_list)):     
    print(f'File name: {files[i]}')
    print('\n')
    check_missing_values(df_list[i])
    print('\n')
    check_duplicates(df_list[i])
    print('\n')
    df_list[i].info()
    print('\n', '\n')

File name: orderlines


Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


Duplicated rows:  0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB

 

File name: orders


Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


Duplicated rows:  0


<class 'pandas.core.frame.DataFrame'

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be of type numerical (floats).
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe the description could be inferred from the product name.
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    * `total_paid` has 5 missing values. We can start by removing them since it's such a small amount, and in the future we can come back to the issue and investigate why these values are missing.
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning `orders`


In [8]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [9]:
orders.isna().any()

order_id        False
created_date    False
total_paid       True
state           False
dtype: bool

In [10]:
# exploring the problematic rows
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


In [11]:
orders.dropna(inplace=True)

In [12]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

In [13]:
orders.to_csv(path + 'orders_cl.csv', index=False)

## Cleaning orderlines

In [14]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [15]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.')

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


In [16]:
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.sort_values(by= 'unit_price_nd',ascending=True).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
77008,1268645,365886,0,1,APP1465,-119.00,2017-06-15 12:48:54,-11900
153217,1422051,431998,0,1,SYN0176,0.00,2017-11-22 17:14:55,000
67563,1251961,357762,0,1,LIBRO,0.00,2017-05-21 18:26:50,000
67585,1251999,357780,0,1,LIBRO,0.00,2017-05-21 19:57:58,000
67626,1252073,357818,0,1,LIBRO,0.00,2017-05-21 23:21:00,000


In [19]:
orderlines['digits'] = orderlines['unit_price_nd'].str[-2:]
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines.sort_values(by= 'integers',ascending=True).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers
77008,1268645,365886,0,1,APP1465,-119.00,2017-06-15 12:48:54,-11900,00,-119
153215,1422045,431994,0,1,CRU0031,0.00,2017-11-22 17:13:28,000,00,0
197635,1489762,460222,0,1,BEL0189,0.00,2017-12-11 19:49:21,000,00,0
197634,1489761,460222,0,1,APP2499,0.00,2017-12-11 19:49:21,000,00,0
153809,1423003,432467,0,1,BOS0058,0.00,2017-11-22 22:28:04,000,00,0


In [20]:
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['digits']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23,23.74


In [21]:
# last step is to transform it to numeric
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.sort_values(by= 'unit_price',ascending=True).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers,new_unit_price
77008,1268645,365886,0,1,APP1465,-119.0,2017-06-15 12:48:54,-11900,00,-119,-119.00
203877,1499322,464189,0,1,CRU0026-2,0.0,2017-12-16 00:48:57,000,00,0,0.00
153860,1423115,432509,0,1,SHE0044,0.0,2017-11-22 22:45:30,000,00,0,0.00
57881,1235064,349545,0,1,LIBRO,0.0,2017-04-25 12:31:31,000,00,0,0.00
57878,1235057,349542,0,1,LIBRO,0.0,2017-04-25 12:27:15,000,00,0,0.00


In [22]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   digits            293983 non-null  object        
 9   integers          293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


In [23]:
orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], 
                axis=1, inplace=True)
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


In [24]:
orderlines.to_csv(path + 'orderlines_cl.csv', index=False)